<a href="https://colab.research.google.com/github/mattn12/ENPH353_Lab5/blob/main/ENPH353_Lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENPH 353 Lab 5 - Character Recogition


## Import Packages

In [ ]:
import string
import random
from random import randint
import cv2
import numpy as np
import os
import math
import re

from PIL import Image, ImageFont, ImageDraw
from collections import Counter
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend

from ipywidgets import interact
import ipywidgets as ipywidgets


## License Plate Generator

In [ ]:
# Run this command only once:

# makes the output folder /content/pictures/
#!mkdir pictures

In [ ]:
# Run this command only once

# Downloads the license plate template to /content/blank_plate.png
#!gdown --id 1Q_FsX2z6DaDCLM91qGZzwvctaW9bN60v

In [ ]:
path = "/content/"

NUMBER_OF_PLATES = 1000

for i in range(0, NUMBER_OF_PLATES):
  # Pick two random letters
  plate_alpha = ""
  for _ in range(0, 2):
      plate_alpha += (random.choice(string.ascii_uppercase))

  # Pick two random numbers
  num = randint(0, 99)
  plate_num = "{:02d}".format(num)
    
  # Write plate to image
  blank_plate = cv2.imread(path+'blank_plate.png')

  # Convert into a PIL image (this is so we can use the monospaced fonts)
  blank_plate_pil = Image.fromarray(blank_plate)

  # Get a drawing context
  draw = ImageDraw.Draw(blank_plate_pil)
  monospace = ImageFont.truetype(font="/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf", 
                                  size=165)
  draw.text(xy=(48, 75),
            text=plate_alpha + " " + plate_num ,
            fill=(255,0,0), font=monospace)

  # Convert back to OpenCV image and save
  blank_plate = np.array(blank_plate_pil)

  # Write license plate to file
  cv2.imwrite(os.path.join(path + "pictures/", 
                            "plate_{}{}.png".format(plate_alpha,plate_num)),
                            blank_plate)

## Authorize Google Drive Access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Retrieve File Names of Plates


In [ ]:
folder = "/content/pictures"
labels_raw = !ls "{folder}"
labels = labels_raw[0].split()
#print(labels_raw)

In [ ]:
def files_in_folder(folder_path):
  '''
  Returns a list of strings where each entry is a file in the folder_path.
  
  Parameters
  ----------
  
  folder_path : str
     A string to folder for which the file listing is returned.
     
  '''
  files_A = !ls "{folder_path}"
  # The files when listed from Google Drive have a particular format. They are
  # grouped in sets of 4 and have spaces and tabs as delimiters.
  
  # Split the string listing sets of 4 files by tab and space and remove any 
  # empty splits.
  files_B = [list(filter(None, re.split('\t|\s', files))) for files in files_A]
  
  # Concatenate all splits into a single sorted list
  files_C = []
  for element in files_B:
    files_C = files_C + element
  files_C.sort()
  
  return files_C

files = files_in_folder(folder)

print(files)

['plate_AA24.png', 'plate_AA37.png', 'plate_AC83.png', 'plate_AC92.png', 'plate_AD06.png', 'plate_AD31.png', 'plate_AD46.png', 'plate_AD85.png', 'plate_AE67.png', 'plate_AF21.png', 'plate_AF77.png', 'plate_AG65.png', 'plate_AG89.png', 'plate_AH87.png', 'plate_AI88.png', 'plate_AK15.png', 'plate_AK70.png', 'plate_AK94.png', 'plate_AL74.png', 'plate_AL85.png', 'plate_AL95.png', 'plate_AM42.png', 'plate_AN56.png', 'plate_AO20.png', 'plate_AO67.png', 'plate_AO94.png', 'plate_AP14.png', 'plate_AP18.png', 'plate_AP70.png', 'plate_AP78.png', 'plate_AP95.png', 'plate_AP97.png', 'plate_AR21.png', 'plate_AR29.png', 'plate_AS28.png', 'plate_AS47.png', 'plate_AT50.png', 'plate_AU00.png', 'plate_AU33.png', 'plate_AV98.png', 'plate_AW93.png', 'plate_AX81.png', 'plate_AY79.png', 'plate_AY97.png', 'plate_BA12.png', 'plate_BB22.png', 'plate_BB70.png', 'plate_BC17.png', 'plate_BC66.png', 'plate_BC85.png', 'plate_BC95.png', 'plate_BC98.png', 'plate_BE08.png', 'plate_BE92.png', 'plate_BJ17.png', 'plate_BJ

## Slice Images

In [ ]:
#cropping settings
top = 100
bottom = 220
left = 47

In [ ]:
value = 0
dataset = []

for plate in files:
  #read and crop plate
  path = f"{folder}/{plate}"
  img = cv2.imread(path, cv2.IMREAD_COLOR)
  img = img[top:bottom,left:]
  
  allchars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

  #slice every letter
  letter1 = [img[:,0:100], allchars.find(path[24])]

  letter2 = [img[:,100:200], allchars.find(path[25])]
               
  letter3 = [img[:,300:400], allchars.find(path[26])]

  letter4 = [img[:,400:500], allchars.find(path[27])]

  dataset.append(letter1)
  dataset.append(letter2)
  dataset.append(letter3)
  dataset.append(letter4)


In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

## Create Datasets for Neural Net

In [ ]:
# Shuffle the dataset
np.random.shuffle(dataset)

# Genereate X and Y datasets
X_dataset_orig = np.array([data[0] for data in dataset])
Y_dataset_orig = np.array([[data[1]] for data in dataset]).T

NUMBER_OF_LABELS = 36
CONFIDENCE_THRESHOLD = 0.01

# Normalize X (images) dataset
X_dataset = X_dataset_orig/255.

# Convert Y dataset to one-hot encoding
Y_dataset = convert_to_one_hot(Y_dataset_orig, NUMBER_OF_LABELS).T



In [ ]:
VALIDATION_SPLIT = 0.2

print("Total examples: {:d}\nTraining examples: {:d}\nTest examples: {:d}".
      format(X_dataset.shape[0],
             math.ceil(X_dataset.shape[0] * (1-VALIDATION_SPLIT)),
             math.floor(X_dataset.shape[0] * VALIDATION_SPLIT)))
print("X shape: " + str(X_dataset.shape))
print("Y shape: " + str(Y_dataset.shape))

Total examples: 4764
Training examples: 3812
Test examples: 952
X shape: (4764, 120, 100, 3)
Y shape: (4764, 36)


## Building Neural Net

In [ ]:
def reset_weights(model):
  for ix, layer in enumerate(model.layers):
      if (hasattr(model.layers[ix], 'kernel_initializer') and 
          hasattr(model.layers[ix], 'bias_initializer')):
          weight_initializer = model.layers[ix].kernel_initializer
          bias_initializer = model.layers[ix].bias_initializer

          old_weights, old_biases = model.layers[ix].get_weights()

          model.layers[ix].set_weights([
              weight_initializer(shape=old_weights.shape),
              bias_initializer(shape=len(old_biases))])

In [ ]:
conv_model = models.Sequential()
conv_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                             input_shape=(120, 100, 3)))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Flatten())
conv_model.add(layers.Dropout(0.5))
conv_model.add(layers.Dense(512, activation='relu'))
conv_model.add(layers.Dense(36, activation='softmax'))

In [ ]:
conv_model.summary()

In [ ]:
LEARNING_RATE = 1e-4
conv_model.compile(loss='categorical_crossentropy',
                   optimizer=optimizers.RMSprop(lr=LEARNING_RATE),
                   metrics=['acc'])

## Reset Weights

In [ ]:
reset_weights(conv_model)

## Train Neural Net

In [ ]:
history_conv = conv_model.fit(X_dataset, Y_dataset, 
                              validation_split=VALIDATION_SPLIT, 
                              epochs=20, 
                              batch_size=16)

In [ ]:
plt.plot(history_conv.history['loss'])
plt.plot(history_conv.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'val loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_conv.history['acc'])
plt.plot(history_conv.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy (%)')
plt.xlabel('epoch')
plt.legend(['train accuracy', 'val accuracy'], loc='upper left')
plt.show()

## Predicting Letters

In [ ]:


# Display images in the training data set. 
def displayImage(index):
  img = X_dataset[index]
  
  img_aug = np.expand_dims(img, axis=0)
  y_predict = conv_model.predict(img_aug)
  
  plt.imshow(img)  
  
  caption = ("Input: " + str(allchars[np.argmax(Y_dataset[index])])
                            + "\nPredicted: " + str(allchars[np.argmax(y_predict)])
                            + "\nConfidence: " + str(np.amax(y_predict)) + "\n") 
  
  plt.text(0.5, 0.5, caption, 
           color='orange', fontsize = 16,
           horizontalalignment='left', verticalalignment='bottom')


interact(displayImage, 
        index=ipywidgets.IntSlider(min=0, max=X_dataset_orig.shape[0],
                                   step=1, value=10))
